### This is to fine tune a LLama3.2 3B model on a custom dataset using Hugging Face's Transformers library.

In [24]:
# for use in google colab to train with the big GPUns
# %%capture
# !pip install unsloth
# # Also get the latest nightly Unsloth!
# !pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# pip install torch --index-url https://download.pytorch.org/whl/cu121

In [25]:
from datasets import load_dataset
import pandas as pd
import random


In [26]:
ds = load_dataset("seandearnaley/sentiment_analysis_sharegpt_json")


In [27]:
df = pd.DataFrame(ds)

In [28]:
data_dict = df['train'].to_dict()

In [29]:
counter = 0
data = []
for key, value in data_dict.items():
    temp = value
    # print(value['conversations'][0]['value'])
    temp['conversations'][0]['value'] = "You are an advanced AI Finanancial Analyst created to perform sentiment analysis on text. Your task is to carefully read the text and analyze the sentiment it expresses towards the potential future stock value of any company mentioned.  Analyze the sentiment of this text and respond with the appropriate JSON:"
    # print(temp['conversations'][0]['value'])
    conversation = {
        "conversations":temp['conversations']
    }
    data.append(conversation)
    counter+=1


print(counter)
print(len(data_dict))


41385
41385


In [30]:
print(len(data))
print(data[5])

41385
{'conversations': [{'from': 'system', 'value': 'You are an advanced AI Finanancial Analyst created to perform sentiment analysis on text. Your task is to carefully read the text and analyze the sentiment it expresses towards the potential future stock value of any company mentioned.  Analyze the sentiment of this text and respond with the appropriate JSON:'}, {'from': 'human', 'value': 'Bot ma umiędzynarodowić K2\nGiełdowa grupa, która wyrosła z agencji kreatywnej, teraz zarabia głównie na oprogramowaniu. Liczy na to, że opracowany przez nią czatbot wyrośnie na jednorożca.'}, {'from': 'gpt', 'value': '{"reasoning": "The article presents a positive outlook on the company\'s future, mentioning its growth from a creative agency to a profitable software-based business. The mention of the chatbot having the potential to become a \'unicorn\' (a startup valued at over $1 billion) suggests high expectations and a positive sentiment towards the company\'s future stock value.", "sentiment"

In [31]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit")

In [32]:
print(data[0]['conversations'][0]['value'])
print(data[0]['conversations'][1]['value'])
print(data[0]['conversations'][2]['value'])

You are an advanced AI Finanancial Analyst created to perform sentiment analysis on text. Your task is to carefully read the text and analyze the sentiment it expresses towards the potential future stock value of any company mentioned.  Analyze the sentiment of this text and respond with the appropriate JSON:
@USAirways Sadly I'm currently booked on one of your flights tomorrow that I will be changing if I can.
{"reasoning": "The sentiment is negative based on the content of the tweet.", "sentiment": -1.0, "confidence": 0.68}


In [33]:
max_seq_length = 0
for i in range(len(data)):
    inp_tokens = tokenizer(data[0]['conversations'][0]['value'], return_tensors="pt")['input_ids'].shape[1]
    hum_tokens = tokenizer(data[0]['conversations'][1]['value'], return_tensors="pt")['input_ids'].shape[1]
    ret_tokens = tokenizer(data[0]['conversations'][2]['value'], return_tensors="pt")['input_ids'].shape[1]
    total = inp_tokens + hum_tokens + ret_tokens
    if total > max_seq_length:
        max_seq_length = total
        print(max_seq_length)

118


In [34]:
from unsloth import FastLanguageModel
import torch
print(torch.__version__)

# max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

ModuleNotFoundError: No module named 'unsloth'

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
alpaca_prompt = """

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)